# OrbitPy TAT-C Example for discrete time coverage calculations

OrbitPy is used to access calculations, followed by TAT-C to coduct coverage analysis.

In [12]:
from joblib import Parallel, delayed

import warnings

import tempfile
import os, shutil
import csv

script_directory = os.path.dirname(os.path.abspath("__file__"))
temp_dir = os.path.join(script_directory, "temp")
os.makedirs(temp_dir, exist_ok=True)
#os.environ['TMPDIR'] = temp_dir


import json
from datetime import datetime, timedelta, timezone
from shapely.geometry import box, mapping
from scipy.stats import hmean
import pandas as pd

from astropy.time import Time as AstroPy_Time

from pydantic import AwareDatetime

from tatc.schemas import Instrument as TATC_Instrument, Satellite as TATC_Satellite, TwoLineElements, Point
from tatc.analysis import collect_orbit_track, OrbitCoordinate, OrbitOutput
from tatc.analysis import (
    collect_multi_observations,
    aggregate_observations,
    reduce_observations,
)

from orbitpy.util import OrbitState as OrbitPy_OrbitState, Spacecraft as OrbitPy_Spacecraft, SpacecraftBus as OrbitPy_SpacecraftBus
from orbitpy.propagator import J2AnalyticalPropagator as OrbitPy_J2AnalyticalPropagator, SGP4Propagator as OrbitPy_SGP4Propagator
from orbitpy.coveragecalculator import GridCoverage as OrbitPy_GridCoverage, find_access_intervals as OrbitPy_find_access_intervals
from orbitpy.grid import Grid as OrbitPy_Grid

from eose.orbits import GeneralPerturbationsOrbitState, Propagator
from eose.satellites import Satellite
from eose.utils import CartesianReferenceFrame, PlanetaryCoordinateReferenceSystem
from eose.targets import TargetPoint
from eose.geometry import Position

from eose.access import (
    AccessSample,
    AccessRecord,
    AccessRequest,
    AccessResponse,
)
from eose.grids import UniformAngularGrid

from instrupy import Instrument as InstruPy_Instrument
from instrupy.basic_sensor_model import BasicSensorModel as InstruPy_BasicSensorModel

from eose.instruments import CircularGeometry, BasicSensor
from eose.datametrics import DataMetricsRequest, BasicSensorDataMetricsInstantaneous, DataMetricsSample, DataMetricsRecord, DataMetricsResponse

pd.set_option('display.max_rows', None)

### Define mission parameters

In [13]:
# define the orbit and the instrument
iss_omm_str = '[{"OBJECT_NAME":"ISS (ZARYA)","OBJECT_ID":"1998-067A","EPOCH":"2024-06-07T09:53:34.728000","MEAN_MOTION":15.50975122,"ECCENTRICITY":0.0005669,"INCLINATION":51.6419,"RA_OF_ASC_NODE":3.7199,"ARG_OF_PERICENTER":284.672,"MEAN_ANOMALY":139.0837,"EPHEMERIS_TYPE":0,"CLASSIFICATION_TYPE":"U","NORAD_CAT_ID":25544,"ELEMENT_SET_NO":999,"REV_AT_EPOCH":45703,"BSTAR":0.00033759,"MEAN_MOTION_DOT":0.00019541,"MEAN_MOTION_DDOT":0}]'
iss_omm = json.loads(iss_omm_str)[0]

basic_sensor = BasicSensor(   id="Atom",
                        mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=100.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

satellites=[
        Satellite(
            id="ISS",
            orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm),
            payloads=[
                basic_sensor
            ]
        )
    ]

targets = UniformAngularGrid(
        delta_latitude=20, delta_longitude=20, region=mapping(box(-180, -50, 180, 50))
    ).as_targets()

mission_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
mission_duration = timedelta(days=7)
propagate_time_step = timedelta(minutes=0.1)

### Run access calculations with OrbitPy to get access-periods at Target ground points.

Access calculations also involve propagation calculations.


In [14]:
def access_orbitpy(request: AccessRequest) -> AccessResponse:

    
    #### Enumerate and convert from EOSE-API satellites to OrbitPy satellite objects. ####
    # (Enumeration generates distinct orbit-instrument pairs for satellites equipped with multiple instruments.)
    OrbitPy_Satellites = []
    for satellite in request.satellites:
        for instru in satellite.payloads: 
            if instru.id in request.payload_ids:
                instru_type = instru.type
                if instru_type == "BasicSensor":
                    # TODO: Update to accept general instrument FOV geometries and orientation
                    instrupy_sensor = InstruPy_Instrument.from_dict({ "@type": "Basic Sensor",
                        "orientation": {"referenceFrame": "SC_BODY_FIXED", "convention": "REF_FRAME_ALIGNED"},
                        "fieldOfViewGeometry": {"shape": "CIRCULAR", "diameter": instru.field_of_view.diameter},
                        "@id": instru.id
                    })
                else:
                    raise ValueError(f"{instru_type} instrument type is not supported. Only 'BasicSensor' instrument type is supported.")
                
                tle = satellite.orbit.to_tle()

                orbit_state = OrbitPy_OrbitState.from_dict({"tle": {
                                                            "tle_line0": "Unknown",
                                                            "tle_line1": tle[0],
                                                            "tle_line2": tle[1]
                                                            }})
                
                # spacecraft bus attributes are presently ignored and set to default. TODO: Consider the spacecraft bus orientation.
                orbitpy_sat_bus = OrbitPy_SpacecraftBus.from_dict({"orientation":{"referenceFrame":"Nadir_pointing", "convention": "REF_FRAME_ALIGNED"}}) 
                sat = OrbitPy_Spacecraft(_id = satellite.id,
                                    orbitState = orbit_state,
                                    spacecraftBus = orbitpy_sat_bus, 
                                    instrument = [instrupy_sensor]
                                )
                warnings.warn("OrbitPy ignores spacecraft bus orientation, and assumes it to be aligned to the geocentric Nadir pointing frame.", UserWarning)                
                OrbitPy_Satellites.append(sat)
    
    #### Format the Target points into OrbitPy Grid object ####
    lon = []
    lat = []
    target_id = [] 
    # iterate through the Target points
    for tp in request.targets:
        if tp.crs == PlanetaryCoordinateReferenceSystem.EPSG_4326 or tp.crs is None:
            lon.append(tp.position[0])
            lat.append(tp.position[1])
            target_id.append(tp.id)
        else:
            raise ValueError(f"{tp.crs} CRS is not supported by OrbitPy. Only 'EPSG_4326' CRS is supported.")
    
    row_to_target_id = {} # Dictionary to map row numbers ('GP index' in OrbitPy) to target_id
    orbitpy_custom_grid = None
    with tempfile.NamedTemporaryFile(mode='w+t', delete=False, dir=temp_dir) as grid_file:
        writer = csv.writer(grid_file)
        writer.writerow(['lat [deg]', 'lon [deg]', 'id'])
        
        for row_num, (lat_val, lon_val, target_id_val) in enumerate(zip(lat, lon, target_id)):
            writer.writerow([lat_val, lon_val, target_id_val])
            row_to_target_id[row_num] = target_id_val
    

    orbitpy_custom_grid = OrbitPy_Grid.from_customgrid_dict({"@type": "customGrid", "covGridFilePath": grid_file.name})

    
    #### run propagation and coverage with OrbitPy ####
    step_size_s = request.time_step.total_seconds()
    if request.propagator != Propagator.J2:
        propagator = OrbitPy_J2AnalyticalPropagator.from_dict({"@type": "J2 ANALYTICAL PROPAGATOR", "stepSize": step_size_s})
    elif request.propagator != Propagator.SGP4:
        propagator = OrbitPy_SGP4Propagator.from_dict({"@type": "SGP4 PROPAGATOR", "stepSize": step_size_s})
    else:
        raise RuntimeError("OrbitPy only supports J2 and SGP4 propagators.")
            
    start_date_dict = {"@type": "GREGORIAN_UT1", "year": request.start.year, "month": request.start.month, "day": request.start.day, "hour": request.start.hour, "minute": request.start.minute, "second": request.start.second + request.start.microsecond / 1000000.0}
    start_date = OrbitPy_OrbitState.date_from_dict(start_date_dict) # assumed that the time scale is UT1
    duration = request.duration.total_seconds() / 86400.0
    
    for orbitpy_sat in OrbitPy_Satellites:

        # run propagation with OrbitPy
        with tempfile.NamedTemporaryFile(mode='w+t', delete=False,dir=temp_dir) as state_cart_file: # store satellite states in a temporary file.
            propagator.execute(orbitpy_sat, start_date, state_cart_file.name, None, duration)
        
            # run access calculations with OrbitPy
            with tempfile.NamedTemporaryFile(mode='w+t', delete=False,dir=temp_dir) as access_fl:
                cov_calc = OrbitPy_GridCoverage(grid=orbitpy_custom_grid, spacecraft=orbitpy_sat, state_cart_file=state_cart_file.name)
                instru_id = orbitpy_sat.get_instrument().get_id()
                cov_calc.execute(instru_id=instru_id, mode_id=None, use_field_of_regard=True, out_file_access=access_fl.name, mid_access_only=False)
                intervals_df = OrbitPy_find_access_intervals(access_fl.name)
                
                
                grouped_intervals = intervals_df.groupby('GP index')
                access_record = [] # record of accesses at each target point
                for gp_index, group in grouped_intervals:
                    # Iterate over each record in the group
                    access_sample = []
                    for index, row in group.iterrows():
                        access_start = request.start + timedelta(seconds=row['Start time index']*step_size_s) 
                        access_duration = timedelta(seconds=row['Duration']*step_size_s) 
                        # form access sample
                        access_sample.append(AccessSample(
                            satellite_id=orbitpy_sat._id,
                            instrument_id=instru_id,
                            start=access_start,
                            duration=access_duration
                        ))
                    # Add access record
                    access_record.append(AccessRecord(
                        target_id=row_to_target_id[gp_index],
                        samples=access_sample
                    ))
    
    # delete the temporary directory
    shutil.rmtree(temp_dir)
    
    return AccessResponse(
            **request.model_dump(exclude="target_records"),
            target_records=access_record
        )

In [ ]:
request = AccessRequest(
    satellites=satellites,
    targets=targets,
    start=mission_start,
    duration=mission_duration,
    propagator=Propagator.SGP4,
    payload_ids=["Atom"]
)

#display(request.model_dump_json())

access_response = access_orbitpy(request)

#display(access_response.model_dump_json())

access_data = access_response.as_dataframe()

#display(access_data)

### Run coverage analysis with TAT-C on the Access response from OrbitPy

In [ ]:
import geopandas as gpd

from eose.coverage import (
    CoverageSample,
    CoverageRecord,
    CoverageRequest,
    CoverageResponse,
)

def coverage_tatc(request: CoverageRequest) -> CoverageResponse:
    aggregated_obs = aggregate_observations(
        gpd.GeoDataFrame(
            [
                {
                    "point_id": request.targets.index(target),
                    "geometry": target.as_geometry(),
                    "satellite": sample.satellite_id,
                    "instrument": sample.instrument_id,
                    "start": sample.start,
                    "end": sample.start + sample.duration,
                    "epoch": sample.start + sample.duration / 2,
                }
                for record in request.target_records
                for target in [t for t in request.targets if t.id == record.target_id]
                for sample in record.samples
                if sample.satellite_id not in request.omit_satellite_ids
                if sample.instrument_id not in request.omit_payload_ids
            ]
        )
    )
    reduced_obs = reduce_observations(aggregated_obs)
    return CoverageResponse(
        **request.model_dump(
            exclude=["target_records", "harmonic_mean_revisit", "coverage_fraction"]
        ),
        target_records=list(
            reduced_obs.apply(
                lambda r: CoverageRecord(
                    **next(
                        record
                        for record in request.target_records
                        if request.targets[r["point_id"]].id == record.target_id
                    ).model_dump(exclude=["samples", "mean_revisit", "number_samples"]),
                    samples=aggregated_obs[
                        aggregated_obs.point_id == r["point_id"]
                    ].apply(
                        lambda s: CoverageSample(
                            start=s.start,
                            duration=s.end - s.start,
                            satellite_id=s.satellite,
                            instrument_id=s.instrument,
                            revisit=None if pd.isna(s.revisit) else s.revisit,
                        ),
                        axis=1,
                    ),
                    mean_revisit=(
                        None
                        if pd.isna(r["revisit"])
                        else timedelta(seconds=r["revisit"].total_seconds())
                    ),
                    number_samples=r["samples"],
                ),
                axis=1,
            )
        )
        + [
            CoverageRecord(target_id=target.id)
            for i, target in enumerate(request.targets)
            if not any(reduced_obs["point_id"] == i)
        ],
        harmonic_mean_revisit=(
            None
            if reduced_obs.dropna(subset="revisit").empty
            else timedelta(
                seconds=hmean(
                    reduced_obs.dropna(subset="revisit")["revisit"].dt.total_seconds()
                )
            )
        ),
        coverage_fraction=len(reduced_obs.index) / len(request.targets),
    )
    
request2 = CoverageRequest(**access_response.model_dump())

display(request2.model_dump_json())

response2 = coverage_tatc(request2)

display(response2.model_dump_json())

data2 = response2.as_dataframe()

display(data2)

In [ ]:
import matplotlib.pyplot as plt

# load shapefile
world = gpd.read_file(
    "https://naciscdn.org/naturalearth/110m/physical/ne_110m_land.zip"
)

# example composite plot using GeoDataFrames
fig, ax = plt.subplots()
ax.set_title(f"Number Samples (Coverage={response2.coverage_fraction:.1%})")
data2.plot(ax=ax, column="number_samples", legend=True)
world.boundary.plot(ax=ax, lw=0.5, color="k")
ax.set_aspect("equal")
plt.show()

# example composite plot using GeoDataFrames
fig, ax = plt.subplots()
ax.set_title(
    f"Mean Revisit (Harmonic Mean={response2.harmonic_mean_revisit/timedelta(hours=1):.1f} hr)"
)
data2["mean_revisit_hr"] = data2.apply(
    lambda r: r["mean_revisit"] / timedelta(hours=1), axis=1
)
data2.plot(ax=ax, column="mean_revisit_hr", legend=True)
world.boundary.plot(ax=ax, lw=0.5, color="k")
ax.set_aspect("equal")
plt.show()